In [1]:
import gzip
import numpy as np
from keras.utils import to_categorical

#Funktion zum Laden der einzelnen Bilder
def open_images(filename):
    #öffnen der gzip-Datein
    with gzip.open(filename, "rb") as file:
        #Laden aller daten
        data = file.read()
        #Schreiben der Daten in ein Numpy Array und Rückgabe des Arrays
        return np.frombuffer(data, dtype=np.uint8, offset=16)\
            .reshape(-1,28,28)\
            .astype(np.float32)
    
def open_labels(filename):
    with gzip.open(filename,"rb") as file:
        data = file.read()
        return np.frombuffer(data,dtype=np.uint8,offset=8)

#erstellen der Daten arrays
X_train = open_images("../Kursmaterialien/data/fashion/train-images-idx3-ubyte.gz")
y_train = open_labels("../Kursmaterialien/data/fashion/train-labels-idx1-ubyte.gz")

X_test = open_images("../Kursmaterialien/data/fashion/t10k-images-idx3-ubyte.gz")
y_test = open_labels("../Kursmaterialien/data/fashion/t10k-labels-idx1-ubyte.gz")

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)


Using TensorFlow backend.
C:\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:529: FutureWarning: Passing (type, 1) or '1type

In [27]:
from keras.models import Sequential
#Import von Convolutional Layer (in 2 Dimensional - oben und unten - Bilder sind Zweidimensional)
#Flatten Layer dient dazu die übereinanderliegenden Conv2D Layer nebeneinander zu legen, um dann diese dem Dense Layer zu 
#übermitteln
from keras.layers import Dense, Conv2D, Flatten

model = Sequential()

#Hinzufuegen eines Layers mit Dense
#100 = Anzahl der Neuronen (frei gewaehlt)
#784 = Anzahl der Pixel pro Bild
#Verschiedene Werte werden ausprobiert um eine höhere Genauigkeit zu erhalten
#meist ist es sigmoid und relu

###Kommentare zu CNN###
#Parameter von Conv2D:
#Anzahl der Filter (Start mit 10)
#kernel_size = größe der Filter (der Matrize)

model.add(Conv2D(30, kernel_size=(3,3), activation="relu", input_shape=(28,28,1))) # Hidden - Layer
model.add(Flatten()) #legt alle entstandenen Ebenen nebeneinander
model.add(Dense(10,activation="softmax")) # Output - Layer - Die 10 repräsentiert die Anzahl der Ausgänge

#1. Parameter = Optimierung der Gewichtung über SGD (Stochastic Gradient Decent - Gradientenabstiegsverfahren)
# andere: rmsprop, adam - ausprobieren welche besser sind
#2. Parameter = Standardwert
model.compile(optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"])

#so lange trainieren, bis sich das loss nicht viel ändert
#ziel ist das loss zu minimieren

model.fit(
    X_train.reshape(60000,28,28,1),
    y_train,
    epochs=80, #Anzahl wie häufig die Epochen durchlaufen werden sollen
    batch_size=1000 #Nach 1000 Bildern werden die Gewichte aktualisiert
)

###Ergebnisse:
#optimizer bisher rmsprop deutlich am besten
#batch_size nicht großartig relevant - Unterschied 100 zu 1000 ca. 1%

Epoch 1/80
60000/60000 [==============================] - 18s 293us/step - loss: 11.5931 - accuracy: 0.2790
Epoch 2/80
60000/60000 [==============================] - 17s 289us/step - loss: 11.3630 - accuracy: 0.2934
Epoch 3/80
60000/60000 [==============================] - 17s 288us/step - loss: 8.9748 - accuracy: 0.4381
Epoch 4/80
60000/60000 [==============================] - 17s 287us/step - loss: 8.2740 - accuracy: 0.4798
Epoch 5/80
60000/60000 [==============================] - 18s 293us/step - loss: 6.0096 - accuracy: 0.6073
Epoch 6/80
60000/60000 [==============================] - 18s 293us/step - loss: 4.6850 - accuracy: 0.6537
Epoch 7/80
60000/60000 [==============================] - 18s 296us/step - loss: 0.3997 - accuracy: 0.8729
Epoch 8/80
60000/60000 [==============================] - 18s 294us/step - loss: 0.2823 - accuracy: 0.9020
Epoch 9/80
60000/60000 [==============================] - 18s 299us/step - loss: 0.2216 - accuracy: 0.9209
Epoch 10/80
60000/60000 [==========

60000/60000 [==============================] - 18s 294us/step - loss: 0.0104 - accuracy: 0.9975
Epoch 77/80
60000/60000 [==============================] - 18s 294us/step - loss: 0.0126 - accuracy: 0.9969
Epoch 78/80
60000/60000 [==============================] - 17s 289us/step - loss: 0.0129 - accuracy: 0.9972
Epoch 79/80
60000/60000 [==============================] - 18s 301us/step - loss: 0.0086 - accuracy: 0.9983
Epoch 80/80
60000/60000 [==============================] - 18s 304us/step - loss: 0.0083 - accuracy: 0.9980


In [3]:
Conv2D?

## Es gilt: Das Netz passt sich an die Daten an

Testgenauigkeit nur minimal schlechter als Trainigsgenauigkeit = Das Modell darf etwas komplexer sein
Testgenauigkeit viel schlechter als Trainingsgenauigkeit = Das Modell ist zu komplex (hat sich zu sehr an Training angepasst)
Beides schleck = Man braucht mehr Daten

In [25]:
# testdaten Genauigkeit vorhersagen

model.evaluate(X_train.reshape(-1,28,28,1),y_train)

60000/60000 [==============================] - 5s 76us/step


[0.28795349276860555, 0.9003333449363708]

In [26]:
model.evaluate(X_test.reshape(-1,28,28,1),y_test)

10000/10000 [==============================] - 1s 77us/step


[0.41013165278434754, 0.8654000163078308]

In [11]:
np.sum(predictions[0])

1.0